In [1]:
# get libs
import os
import requests
from bs4 import BeautifulSoup
import openai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [2]:
# set url
url = "https://www.butterbeready.com/seafood-boil-with-garlic-butter-sauce/"

In [3]:
# get texts
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    print(f"Status code: {response.status_code}")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

Status code: 200


In [4]:
# extract and print the text from each paragraph
texts_from_url = []

for p in paragraphs:
    texts_from_url.append(p.get_text())

In [19]:
# flatten
flatten_texts_from_url = ' '.join(texts_from_url)

In [5]:
# instantiate openai client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [38]:
# get recipe summarizer
my_assistant = client.beta.assistants.retrieve(os.getenv("OPENAI_ASST_ID"))
print(my_assistant)

Assistant(id='asst_1VheFuNiBaFWHIuP3gBQAkPp', created_at=1699510971, description=None, file_ids=[], instructions='You are a helpful recipe summarizer. The user will give you raw texts taken from a recipe website, and you will analyze the texts and succinctly extract the recipe information, including ingredients and steps, presenting them in a clear and concise format.\n\nBe sure in your response to include measurements of each ingredient, so that the user knows how much each ingredient is needed.\n\nIgnore non recipe related texts, such as comments, picture captions, etc.\n', metadata={}, model='gpt-4-1106-preview', name='Recipe Summarizer', object='assistant', tools=[])


In [39]:
# start thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_pbL3l0Dt50V2GG3Yijre8qGA', created_at=1699513106, metadata={}, object='thread')


In [41]:
# send user message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"Raw recipe tex: ```{flatten_texts_from_url}```"
)

In [42]:
# run assistant
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=my_assistant.id,
)

In [46]:
# check run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [47]:
# get response
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [48]:
# %% cell 13
print("Response from assistant:")
last_assistant_message = None
for message in messages.data:
    if message.role == "assistant":
        last_assistant_message = message

if last_assistant_message:
    print(f"Assistant: {last_assistant_message.content[0].text.value}")

Response from assistant:
Assistant: The provided text doesn't include the complete recipe details such as the specific ingredients and their measurements, nor the step-by-step instructions for preparing the Cajun Seafood Boil and the Garlic Butter Sauce. The text mostly consists of a description and commentary on the dish, with references to a detailed recipe card that is not included in the text.

Based on the text, here's a basic summary of what the dish entails, but please note that the details such as ingredient quantities and cooking times are missing:

Ingredients:
- Jumbo shrimp
- Crab (snow crab clusters or other varieties as preferred; ensure pre-cooked if applicable)
- Andouille sausage
- Potatoes
- Eggs (hard-boiled)
- Corn on the cob
- Cajun spices
- Aromatics
- Beer (lagers, IPAs, or pilsners) or a substitute if preferred
- Garlic Butter Sauce (details not provided)

Steps:
1. Combine the seafood, sausage, potatoes, corn, and eggs with the Cajun spices and aromatics in a f